# LitMatter Rapid Prototyping
* This notebook shows how to experiment with Graph Neural Networks using the LitMatter template.  
* In this example, we train a [DimeNet](https://arxiv.org/abs/2003.03123) model on the [QM9](https://www.nature.com/articles/sdata201422) dataset.  
* The training workflow shown here can be scaled to hundreds of GPUs by changing a single keyword argument!

In [ ]:
import numpy as np
import os

from torch_geometric.datasets import QM9
from torch_geometric.nn import DimeNet

from pytorch_lightning.metrics import Accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import (LightningDataModule, LightningModule, Trainer,
                               seed_everything)

from lit_models.models import LitDimeNet
from lit_data.data import LitQM9

In [ ]:
seed_everything(11)

### Load data
Datasets are organized as a `pytorch_lightning.LightningDataModule`. All data downloading, processing, and transformations are defined in the `DataModule`. For simplicity, here we use the QM9 dataset available through PyTorch Geometric.

In [ ]:
dataset = QM9('data/QM9')
target = 0
_, datasets = DimeNet.from_qm9_pretrained('data/QM9', dataset, target)
datamodule = LitQM9(datasets)
datamodule.setup()

### Set up model
The model is defined in a `pytorch_lightning.LightningModule`. Any PyTorch or PyTorch Geometric code can be used to build the model.

In [ ]:
model = LitDimeNet(target)

In [ ]:
model.summarize()

In [ ]:
# add any desired callbacks
checkpoint_callback = ModelCheckpoint(monitor='val_loss', save_top_k=1)

In [ ]:
trainer = Trainer(gpus=-1,  # use all available GPUs on each node
                  num_nodes=1,  # change to number of available nodes
                 accelerator='ddp',
                 max_epochs=5,
                 callbacks=[checkpoint_callback],
                 )

### Model training and evaluation

In [ ]:
trainer.fit(model, datamodule=datamodule)

In [ ]:
trainer.test()

That's it! By changing the `num_nodes` argument, training can be distributed across all available GPUs. For longer training jobs on an HPC cluster, see the provided example batch scripts.